# Learning from Yellow Taxi Data

In [14]:
import random
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

### Load the data

In [25]:
taxidata = pd.read_csv("./data/2016-01.csv", header=0)

### Define train and test sets

In [22]:
itrain, itest = train_test_split(xrange(taxidata.shape[0]), train_size=0.8)
mask=np.ones(taxidata.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)
mask[:10]

array([ True,  True, False,  True,  True,  True,  True,  True,  True, False], dtype=bool)

### Drop all of the columns except for the pickup and dropoff

In [40]:
X = taxidata[["pickup_longitude", "pickup_latitude"]]
y = taxidata[["dropoff_longitude", "dropoff_latitude"]]

In [45]:
Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
n_samples = Xtrain.shape[0]
n_features = Xtrain.shape[1]

print Xtrain.shape
max_samples = 1000000
if Xtrain.shape[0] > max_samples:
    rows = random.sample(Xtrain.index, max_samples)
    Xtrain = Xtrain.ix[rows]
    ytrain = ytrain.ix[rows]
print Xtrain.shape

(8725486, 2)
(1000000, 2)


In [5]:
estimator = RandomForestRegressor(n_estimators=20, n_jobs=-1)